In [4]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_11108/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [1]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [2]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_11108/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_49_7,0.999515,0.962215,0.996318,0.999666,0.000248,0.022431,0.002580,0.000122,0.003445,0.015745,1.003515,0.016297,82.605023,128.844537,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_49_3,0.999489,0.961850,0.996393,0.999736,0.000261,0.022648,0.002527,0.000096,0.003450,0.016167,1.003706,0.016735,82.499102,128.738616,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_49_0,0.999456,0.961385,0.996464,0.999784,0.000278,0.022923,0.002477,0.000079,0.003493,0.016671,1.003941,0.017257,82.376247,128.615760,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_48_7,0.999428,0.961070,0.996466,0.999729,0.000292,0.023111,0.002476,0.000068,0.003573,0.017101,1.004146,0.017701,82.274517,128.514031,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_48_3,0.999384,0.960482,0.996406,0.999749,0.000315,0.023459,0.002517,0.000063,0.003682,0.017743,1.004464,0.018366,82.126985,128.366498,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,model_6_46_4,0.991486,-3.052455,0.002066,0.964241,0.004353,2.405712,0.699074,0.012264,0.025766,0.065980,0.938275,0.068296,60.873596,95.903530,"Hidden Size=[4, 2], regularizer=0.02, learning..."
1493,model_6_46_2,0.991556,-3.073891,-0.009460,0.967524,0.004318,2.418438,0.707147,0.011138,0.025614,0.065710,0.938780,0.068016,60.890026,95.919961,"Hidden Size=[4, 2], regularizer=0.02, learning..."
1494,model_6_47_5,0.993585,-3.139842,0.004975,0.984422,0.003280,2.457589,0.697035,0.010444,0.019323,0.057274,0.953490,0.059284,61.439647,96.469582,"Hidden Size=[4, 2], regularizer=0.02, learning..."
1495,model_6_45_7,0.990452,-3.277119,-0.096379,0.974299,0.004882,2.539082,0.768036,0.016271,0.024339,0.069874,0.930776,0.072326,60.644271,95.674206,"Hidden Size=[4, 2], regularizer=0.02, learning..."
